In [1]:
import networkx as nx
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("lib/datasets/twitch_gamers/large_twitch_edges.csv")
df

,numeric_id_1,numeric_id_2
0,98343,141493
1,98343,58736
2,98343,140703
3,98343,151401
4,98343,157118
...,...,...
6797552,97507,29359
6797553,71175,12020
6797554,151702,128281
6797555,118034,38021


In [3]:
g = nx.from_pandas_edgelist(df, source='numeric_id_1', target="numeric_id_2", edge_attr=None, create_using=nx.DiGraph())

In [4]:
personal = {}
query_node = 1
for i in range(g.number_of_nodes()):
    if i == query_node:
        personal[i] = 1
    else:
        personal[i] = 0

In [5]:
ppr = nx.pagerank(g,personalization= personal)

In [6]:
top_ppr = sorted(ppr, key=ppr.get, reverse=True)[:200]

In [8]:
tmp = []
tmp1 = []
idk = df["numeric_id_1"].unique()
for i in top_ppr:
    if i in idk:
        tmp.append("Hub "+str(i))
        tmp1.append(i)
con_ppr = tmp1

In [9]:
unique = []
for i in con_ppr:
    group = df[df.numeric_id_1 == i]["numeric_id_2"]
    for j in group:
        unique.append(j)
unique = list(set(unique))

Apparently the graph is not connected. I'll need to figure this out another time

In [10]:
from networkx.algorithms import bipartite
b = nx.Graph()
b.add_nodes_from(tmp, bipartite = 0)
b.add_nodes_from(unique,bipartite=1)

In [11]:
for i in range(len(con_ppr)):
    group = df[df.numeric_id_1 == con_ppr[i]]["numeric_id_2"]
    for j in group:
        b.add_edge(tmp[i],j)

In [12]:
nx.is_connected(b)

False

In [27]:
#from networkx.algorithms.components.connected import connected_component_subgraphs
test = True
tmp = []
hubs = []
auth = []
for i in nx.connected_components(b):
    if test:
        test= False
        for j in i:
            if type(j) == str:
                hubs.append(j)
            else:
                auth.append(j)
bi = nx.Graph()
bi.add_nodes_from(hubs, bipartite = 0)
bi.add_nodes_from(auth, bipartite = 1)
for i in hubs:
    curr = int(i[4:])
    group = df[df.numeric_id_1 == curr]["numeric_id_2"]
    for j in group:
        bi.add_edge(i, j)
nx.is_connected(bi)

True

In [47]:
from networkx.algorithms.bipartite.matrix import biadjacency_matrix
L = biadjacency_matrix(bi, row_order = hubs, column_order=auth).toarray()
L.shape

(124, 6574)

In [56]:
Lr = []
for i in L:
    Lr.append(i / i.sum())
Lr = np.array(Lr)
Lc = []
for i in L.T:
    Lc.append(i / i.sum())
Lc = np.array(Lc)
A = np.dot(Lc, Lr)


array([1., 1., 1., ..., 1., 1., 1.])

In [57]:
evals, evecs = np.linalg.eig(A.T)
evec1 = evecs[:,np.isclose(evals, 1)]

#Since np.isclose will return an array, we've indexed with an array
#so we still have our 2nd axis.  Get rid of it, since it's only size 1.
evec1 = evec1[:,0]

stationary = evec1 / evec1.sum()

#eigs finds complex eigenvalues and eigenvectors, so you'll want the real part.
stationary = stationary.real
max(stationary)

array([0.00013805, 0.00013805, 0.00013805, ..., 0.00013805, 0.00013805,
       0.00027609])

In [62]:
idx = (-stationary).argsort()[:5]
# Top 5 Predicted channels
np.array(auth)[idx]

array([71519, 62246, 33481, 45995, 62080])

In [68]:
for i in idx:
    try:
        print(nx.shortest_path(g, source=1, target=i))
    except:
        print("No shortest path")

[1, 19719, 63663, 90749, 1343]
No shortest path
[1, 100476, 18316, 166]
No shortest path
[1, 19719, 37185, 731, 5862]


In [49]:
from operator import itemgetter
from networkx.algorithms.bipartite.matrix import biadjacency_matrix
node_list = list(b.nodes)
node_list.sort()
bi_adj = nx.adjacency_matrix(b,node_list)
bi_adj.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)